# Importing all the important libraries

In [1]:
import tensorflow as tf
import numpy as np
import h5py
import torch
from tqdm import tqdm
from utils.dataframe import (
    convert_to_hf,
    load_model_variants_df,
    save_model_variants_hf
)
from transformers import (
    AutoTokenizer, 
)
from IPython.display import display
import tensorflow_hub as hub
import tensorflow_text as text
import fasttext
import fasttext.util
from laserembeddings import Laser
from utils.gpu import get_device
from sklearn.decomposition import PCA

# Set Settings

In [2]:
tqdm.pandas()

# Common

In [3]:
# gpu device 
device = get_device()


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details:  {'device_name': 'METAL'}
Using device: mps


# 2. Enhance Burmese Contextual Representations
- Use LASER, mUSE, and FastText for cross-lingual and morphology-aware training.
- Fine-tune mBERT, XLM-R on Burmese dataset after adding contextual embeddings.
- Train models again using combined embeddings.

In [4]:
# function to get fine tuned tokenizer
def get_fine_tuned_tokenizer(model_name, spt_name):
    # Load LoRA Weights
    lora_checkpoint_path = f"model-variants/models/{model_name}_{spt_name.upper()}"

    # Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(lora_checkpoint_path)

    return tokenizer

## Data Preprocessing

In [5]:
# LASER Embeddings
laser = Laser()

In [6]:
# mUSE (Multilingual Universal Sentence Encoder)
muse = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

2025-02-19 18:31:54.010274: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-02-19 18:31:54.010304: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2025-02-19 18:31:54.010308: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
I0000 00:00:1739957514.010328 1266786 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1739957514.010353 1266786 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-02-19 18:31:56.132603: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [7]:
# FastText for Morphology-Aware Training
fasttext.util.download_model('my', if_exists='ignore')  # Download Burmese FastText model
fasttext_model = fasttext.load_model('cc.my.300.bin')

In [8]:
# load data
dataset = load_model_variants_df("combined")

# split into train and test
train_df = dataset.sample(frac=0.8, random_state=42)
test_df = dataset.drop(train_df.index)

# conert to hugging face dataset
train_dataset = convert_to_hf(train_df)
test_dataset = convert_to_hf(test_df)

### Generate Embeddings

In [9]:
# Extract sentences from DataFrame
all_train_sentences = train_df["source"].tolist()

In [15]:
batch_size = 256
all_train_embeddings = []

In [ ]:
# generate embeddings
for i in tqdm(range(0, len(all_train_sentences), batch_size), desc="Processing batches"):
    batch = all_train_sentences[i:i + batch_size]

    # Generate embeddings on GPU
    batch_tensor = tf.convert_to_tensor(batch)  # Convert to TensorFlow tensor
    embeddings_muse = muse(batch_tensor).numpy()  # GPU-accelerated mUSE embeddings

    # Generate FastText embeddings
    embeddings_fasttext = np.array([
        fasttext_model.get_sentence_vector(sentence.replace("\n", " ").strip())  # Remove newlines
        for sentence in batch
    ])

    # Generate LASER embeddings
    embeddings_laser = laser.embed_sentences(batch, lang="my")

    # Stack all embeddings
    combined_embeddings = np.hstack([embeddings_laser, embeddings_muse, embeddings_fasttext])
    all_train_embeddings.append(combined_embeddings)

Processing batches:   2%|▏         | 80/5087 [01:33<1:41:24,  1.22s/it]

In [ ]:
# Convert to NumPy array
all_train_embeddings = np.concatenate(all_train_embeddings, axis=0)
print(f"Finished computing embeddings! Shape: {all_train_embeddings.shape}")

In [ ]:
print("Applying PCA for dimensionality reduction...")

pca = PCA(n_components=300)
pca.fit(all_train_embeddings)  # Fit PCA on training set

In [ ]:
# Transform training embeddings
all_train_embeddings_pca = pca.transform(all_train_embeddings)
print(f"PCA applied! Reduced shape: {all_train_embeddings_pca.shape}")

In [ ]:
print("Saving embeddings and sentences to HDF5 format...")

with h5py.File("model-variants/data/contextual_train_embeddings.h5", "w") as hf:
    hf.create_dataset("embeddings", data=all_train_embeddings_pca)
    hf.create_dataset("sentences", data=np.array(all_train_sentences, dtype=h5py.string_dtype(encoding="utf-8")))

print("Data saved successfully!")

### Load Embeddings

In [ ]:
# Load the HDF5 file
with h5py.File("model-variants/data/contextual_train_embeddings.h5", "r") as hf:
    all_train_embeddings_pca = np.array(hf["embeddings"])  # Load PCA-transformed embeddings
    all_train_sentences = [s.decode("utf-8") for s in hf["sentences"]]  # Decode stored UTF-8 sentences

#

In [ ]:
# Debugging Output
print(f"✅ Loaded PCA Embeddings Shape: {all_train_embeddings_pca.shape}")  # Should be (num_samples, 300)
print(f"✅ Loaded Sentences: {len(all_train_sentences)} samples")  # Should match the number of embeddings
print(f"🔍 Example Sentence: {all_train_sentences[0]}")
print(f"🔍 Example Embedding (first 5 values): {all_train_embeddings_pca[0][:5]}")

### Tokenize

In [ ]:
@torch.no_grad()
def generate_contextual_embeddings(sentences):
    """Generates contextual embeddings using LASER, FastText, and mUSE."""
    sentences = [sentence.replace("\n", " ").strip() for sentence in sentences]

    # Generate embeddings on GPU
    batch_tensor = tf.convert_to_tensor(sentences)
    embeddings_muse = muse(batch_tensor).numpy()  # GPU-accelerated mUSE embeddings

    embeddings_fasttext = np.array([
        fasttext_model.get_sentence_vector(sentence) for sentence in sentences
    ])  # FastText embeddings

    embeddings_laser = laser.embed_sentences(sentences, lang="my")  # LASER embeddings

    # Apply PCA transformation
    combined_embeddings = np.hstack([embeddings_laser, embeddings_muse, embeddings_fasttext])
    contextual_embeddings = pca.transform(combined_embeddings)

    return contextual_embeddings


In [ ]:
@torch.no_grad()
def tokenize_with_contextual_embeddings(examples, tokenizer):
    """Tokenizes input sentences while integrating contextual embeddings."""
    sentences = examples["source"]

    # Generate contextual embeddings
    contextual_embeddings = generate_contextual_embeddings(sentences)
    contextual_embeddings = torch.tensor(contextual_embeddings, dtype=torch.bfloat16).to(tokenizer.device)

    # Tokenization
    tokenized_input = tokenizer(sentences, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    tokenized_output = tokenizer(examples["target"], truncation=True, padding="max_length", max_length=512, return_tensors="pt")

    batch_size = tokenized_input["input_ids"].shape[0]
    seq_len = tokenized_input["input_ids"].shape[1]
    hidden_size = contextual_embeddings.shape[-1]

    # Expand contextual embeddings for each token
    contextual_embeddings = contextual_embeddings.unsqueeze(1).expand(batch_size, seq_len, hidden_size).to(tokenized_input["input_ids"].device)

    return {
        "input_ids": tokenized_input["input_ids"],
        "attention_mask": tokenized_input["attention_mask"],
        "labels": tokenized_output["input_ids"],
        "contextual_embeds": contextual_embeddings
    }

In [ ]:
def prepare_contextual_embedding_dataset(model_name, spt_name):
    """
    Tokenizes datasets with contextual embeddings for fine-tuning XLM-R & mBERT.
    Saves processed train and test datasets with embeddings.
    """
    print(f"Preparing dataset for {model_name} with {spt_name} embeddings...")

    # Load tokenizer
    tokenizer = get_fine_tuned_tokenizer(model_name, spt_name)

    # Process Training Dataset
    print("⚙️ Tokenizing train dataset with contextual embeddings...")
    train_tokenized = train_dataset.map(
        lambda x: tokenize_with_contextual_embeddings(x, tokenizer),
        batched=True,
        num_proc=8,  # Parallel processing for speed
        load_from_cache_file=False,
        desc=f"Tokenizing train dataset for {model_name} with {spt_name} (Contextual Embeddings)",
    )

    # Save train dataset
    train_filename = f"contextual_embedded_{model_name.lower()}_{spt_name}_train"
    save_model_variants_hf(train_tokenized, train_filename)
    print(f"✅ Saved processed train dataset as {train_filename}")

    # Process Test Dataset
    print("⚙️ Tokenizing test dataset with contextual embeddings...")
    test_tokenized = test_dataset.map(
        lambda x: tokenize_with_contextual_embeddings(x, tokenizer),
        batched=True,
        num_proc=8,  # Parallel processing
        load_from_cache_file=False,
        desc=f"Tokenizing test dataset for {model_name} with {spt_name} (Contextual Embeddings)",
    )

    # Save test dataset
    test_filename = f"contextual_embedded_{model_name.lower()}_{spt_name}_test"
    save_model_variants_hf(test_tokenized, test_filename)
    print(f"✅ Saved processed test dataset as {test_filename}")

    print(f"🎉 Finished processing dataset for {model_name} with {spt_name}")

#### mBERT

In [ ]:
# create embedding with bpe
prepare_contextual_embedding_dataset("mBERT", "bpe")

In [ ]:
# create embedding with unigram
prepare_contextual_embedding_dataset("mBERT", "unigram")

#### XLM-R

In [ ]:
# create embedding with bpe
prepare_contextual_embedding_dataset("XLM-R", "bpe")

In [ ]:
# create embedding with unigram
prepare_contextual_embedding_dataset("XLM-R", "unigram")